In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import csv
from sklearn.decomposition import PCA


2023-12-01 17:12:43.613900: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-01 17:12:43.615307: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-01 17:12:43.634566: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 17:12:43.634586: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 17:12:43.635164: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

### Data cleaning

1. Deletes the 'Id' column and checks for common columns in data frame 
2. Then drops columns that have more than 20% of its values empty
3. Apply min/Max Scaling
4. Test Train Split the data

In [57]:
df = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')


del df['Id']                                                #Delete exdex column
df = df.dropna(thresh=len(df) - int(0.1*len(df)), axis=1)   ##Allows for 20% of the columns to be empty, more than that are dropped
for col in df:
    if  df[col].dtype  == object:
        df[col].fillna(df[col].mode()[0], inplace=True)  ##handles NaN data by populating it 
    else:
        df[col].fillna(df[col].median(), inplace=True)

    if  df[col].dtype  == object:  
        df[col] = pd.Categorical(df[col], categories=df[col].unique()).codes ##maps string to intger values
        df[col] = df[col].astype(int)
                                 
scale11 = df.min()
scale22 = df.max()
df=(df-scale11)/(scale22-scale11)

# y = df['SalePrice']
# del df['SalePrice']  
# scale1 = df.min()
# scale2 = df.max()
# #df=(df-scale1)/(scale2-scale1)
# col_names = np.intersect1d(df.columns, df2.columns)


# X = df[col_names].copy()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, random_state = 0)


In [58]:
X_train, X_test = train_test_split(df, test_size = 0.05, random_state = 0)
X_train = tfdf.keras.pd_dataframe_to_tf_dataset(X_train,label='SalePrice',task=tfdf.keras.Task.REGRESSION)
X_test = tfdf.keras.pd_dataframe_to_tf_dataset(X_test,label='SalePrice',task=tfdf.keras.Task.REGRESSION)
#X_train, X_test = train_test_split(X, test_size = 0.05, random_state = 0)

### Attempt PCA on the data

1. We can reduce the redundant features, around 30 should be sufficient
2. We will use the same decomposition on the test data 

In [59]:
# pca = PCA(n_components=len(col_names)-1)
# pca.fit(X_train)

# plt.plot(np.cumsum(pca.explained_variance_ratio_))

# NFEATURES = 30
# pca = PCA(n_components=NFEATURES)
# X_pca_train = pca.fit_transform(X_train)
# X_pca_test = pca.fit_transform(X_test)

# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# plt.xlabel('Number of components')
# plt.ylabel('Cumulative explained variance')

## Define model to train

1. We Do a random forest model for the regressor
2. Convert data to tfdata set for training
3. Evaluate on the test data

In [60]:
tuner = tfdf.tuner.RandomSearch(num_trials=50, use_predefined_hps=True)
model = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION,tuner=tuner)
model.fit(X_train,verbose=2)
model.evaluate(X_test)
inspector = model.make_inspector()
inspector.evaluation()

Use /tmp/tmpb3m9f_o_ as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'MSSubClass': <tf.Tensor 'data:0' shape=(None,) dtype=float64>, 'MSZoning': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'LotArea': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'Street': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'LotShape': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'LandContour': <tf.Tensor 'data_5:0' shape=(None,) dtype=float64>, 'Utilities': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>, 'LotConfig': <tf.Tensor 'data_7:0' shape=(None,) dtype=float64>, 'LandSlope': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'Neighborhood': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>, 'Condition1': <tf.Tensor 'data_10:0' shape=(None,) dtype=float64>, 'Condition2': <tf.Tensor 'data_11:0' shape=(None,) dtype=float64>, 'BldgType': <tf.Tensor 'data_12:0' shape=(None,) dtype=float64>, 'HouseStyle': <tf.Tensor 'data_13:0'

[INFO 23-12-01 18:28:02.1200 CET kernel.cc:771] Start Yggdrasil model training
[INFO 23-12-01 18:28:02.1200 CET kernel.cc:772] Collect training examples
[INFO 23-12-01 18:28:02.1201 CET kernel.cc:785] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: NUMERICAL
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-12-01 18:28:02.1202 CET kernel.cc:391] Number of batches: 2
[INFO 23-12-01 18:28:02.1202 CET kernel.cc:392] Number of examples: 1387
[INFO 23-12-01 18:28:02.1205 CET kernel.cc:792] Training dataset:
Number of records: 1387
Number of columns: 73

Number of columns by type:
	NUMERICAL: 73 (100%)

Columns:

NUMERICAL: 73 (100%)
	0: "1stFlrSF" NUMERICAL mean:0.190383 min:0 max:1 sd:0.0882839
	1: "2ndFlrSF" NUMERICAL mean:0.168134 min:0 max:0.906538 sd:0.210677
	2: "3SsnPorch" NUMERICAL mea

Model trained in 3:55:53.449395
Compiling model...
Model compiled.


1/1 [==============================] - 0s 83ms/step - loss: 0.0000e+00


Evaluation(num_examples=1387, accuracy=None, loss=None, rmse=0.03921117706525829, ndcg=None, aucs=None, auuc=None, qini=None)


### Cleaning test data

1. Delete index column
2. Only use features that were used on the trained data - saved in column_names
3. Use same scaling as done train data

In [30]:

idx = df2['Id']
df_data = df2[col_names].copy()

for col in df_data:
    if  df_data[col].dtype  == object:
        df_data[col].fillna(df_data[col].mode()[0], inplace=True)  ##handles NaN data by populating it 
    else:
        df_data[col].fillna(df_data[col].median(), inplace=True)
    if  df_data[col].dtype  == object:  
        df_data[col] = pd.Categorical(df_data[col], categories=df_data[col].unique()).codes ##maps string to intger values
        df_data[col] = df_data[col].astype(int)

df_data=(df_data-scale1)/(scale2-scale1)
X_pca_eval = pca.fit_transform(df_data)

### Prediction

1. Make a prediction with the model
2. Scale the data back to the value
3. Save data in csv file

In [33]:
prediction = model.predict(X_pca_eval)
s1 = scale11['SalePrice']
s2 = scale22['SalePrice']
y1 = []
for x in prediction:
    y1.append(x[0]*(s2-s1) + s1)              # Back scaling the data

rows=zip(idx,y1)
with open('results.csv', "w") as f:
    writer = csv.writer(f)
    writer.writerow(['Id','SalePrice'])
    for row in rows:
        writer.writerow(row)
    


46/46 [==============================] - 0s 698us/step
